In [1]:
import pandas as pd
import os
from pypdf import PdfReader
import numpy as np
import random
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader

/Users/inesbenhamza/Desktop/untitled folder 2/hw1DL/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#!pip install pypdf
#!pip install sentence-transformers

# directory check 

In [3]:
import os, sys

current_dir = os.getcwd()
print(f"Current working directory: {current_dir}")

project_root = None

if os.path.basename(current_dir) == 'notebooks':
    project_root = os.path.dirname(current_dir)
    print(f"Method 1: Found project root (from notebooks dir): {project_root}")
else:

    if os.path.exists(os.path.join(current_dir, 'helpers')):
        project_root = current_dir
        print(f"Method 2: Found project root (helpers in current dir): {project_root}")
    elif os.path.exists(os.path.join(os.path.dirname(current_dir), 'helpers')):
        project_root = os.path.dirname(current_dir)
        print(f"Method 3: Found project root (helpers in parent dir): {project_root}")
    else:
        
        parent_dir = os.path.dirname(current_dir)
        if os.path.exists(os.path.join(parent_dir, 'helpers')):
            project_root = parent_dir
            print(f"Method 4: Found project root (going up one level): {project_root}")
        else:
            known_project_root = "/Users/inesbenhamza/Downloads/deeplearningproject"
            if os.path.exists(os.path.join(known_project_root, 'helpers')):
                project_root = known_project_root
                print(f"Method 5: Using known absolute path: {project_root}")

if project_root and os.path.exists(os.path.join(project_root, 'helpers')):
    if project_root not in sys.path:
        sys.path.insert(0, project_root)


Current working directory: /Users/inesbenhamza/Downloads/deeplearningproject/notebooks
Method 1: Found project root (from notebooks dir): /Users/inesbenhamza/Downloads/deeplearningproject


In [4]:
#!pip install ollama


In [5]:
#! pip install nltk

In [6]:
#!pip install sentence-transformers
#!pip install torch
#!pip install numpy
#!pip install scipy
#!pip install pandas
#!pip install pyyaml
#!pip install streamlit
#!pip install pymupdf
#!pip install pdfplumber
#!pip install anthropic
#!pip install openai
#!pip install google-generativeai
#!pip install watchdog
#!pip install altair

In [7]:
#!pip install -U sentence-transformers
#!pip install pypdf
#!pip install transformers
#!pip install pandas


In [8]:
#from sentence_transformers import SentenceTransformer
#from SkillsEmbeddings import SkillEmbeddingModel, save_skills_emb

#model = SentenceTransformer("BAAI/bge-m3")
#skill_model = SkillEmbeddingModel(model)

#save_skills_emb(
    #input_csv="skills/skillscleaned.csv",
    #output_npz="skills/skills_encoded.npz",
    #model=skill_model)

# extracting pdf text for encoding 

In [9]:
from pypdf import PdfReader
def extract_text(filename: str):
    reader = PdfReader(filename)
    complete_text = ""
    
    for page in reader.pages:
        complete_text += f"\n{page.extract_text()}"
    return complete_text

In [10]:
if 'project_root' not in globals():
    current_dir = os.getcwd()
    if os.path.basename(current_dir) == 'notebooks':
        project_root = os.path.dirname(current_dir)
    else:
        project_root = current_dir

data_dir = os.path.join(project_root, "data")
print(os.listdir(data_dir))

['HCK_HEC_LANG.csv', '.DS_Store', 'HCK_HEC_XP.csv', 'HCK_HEC_SKILLS.csv', 'HCK_HEC_USER.csv', 'HCK_HEC_STAFFING.csv', 'raw']


# loading all file at the same time not so useful 

# cleaning the lang dataset 
it contains the user IDs, the langauge and the skill level
there are some NAs values that we replace by -1 (need to check why we did that in the first place)  

In [11]:
csvfile = pd.read_csv("/Users/inesbenhamza/Downloads/deeplearningproject/data/HCK_HEC_LANG.csv")


In [12]:
print(csvfile[csvfile['USER_ID']==2433141])
csvfile['LANGUAGE_SKILL_LVL'].isna().sum()

     USER_ID LANGUAGE_SKILL_DSC  LANGUAGE_SKILL_LVL
38   2433141             French               100.0
102  2433141            English               100.0


np.int64(8)

In [13]:
csvfile['LANGUAGE_SKILL_LVL'] = csvfile['LANGUAGE_SKILL_LVL'].fillna(-1) # to handle missing values we fill them with -1

In [14]:
csvfile['LANGUAGE_SKILL_LVL'].isna().sum()


np.int64(0)

In [15]:
df_filtered = csvfile.loc[csvfile.groupby(['USER_ID', 'LANGUAGE_SKILL_DSC'])['LANGUAGE_SKILL_LVL'].idxmax()]
 

df_filtered = df_filtered.reset_index(drop=True)

print(df_filtered.head(10))

   USER_ID LANGUAGE_SKILL_DSC  LANGUAGE_SKILL_LVL
0  2432194            English               100.0
1  2432194             French               100.0
2  2433083            English                80.0
3  2433083             French               100.0
4  2433092            English                80.0
5  2433092             French               100.0
6  2433099            English                80.0
7  2433099             French               100.0
8  2433109            English                80.0
9  2433109             French               100.0


In [16]:
df_filtered[df_filtered['USER_ID'] == 2664664]

,USER_ID,LANGUAGE_SKILL_DSC,LANGUAGE_SKILL_LVL
170,2664664,English,-1.0
171,2664664,French,-1.0


In [17]:
print (df_filtered.duplicated().sum())

0


# storing lang clean into a csv 

In [18]:
df_filtered.to_csv("langcleaned.csv", index=False)


create a consultant_profile_df by merging all csv ( should we creat col skills 1, skills 2 etc ? or skilss : skills1,skills2 etc)

In [19]:
# gettign workign directory 
#os.getcwd()

In [20]:
expyears  = pd.read_csv("/Users/inesbenhamza/Downloads/deeplearningproject/data/HCK_HEC_USER.csv")
expyears['USER_ID'].nunique()

print(expyears.head())

   USER_ID LAST_NAME FIRST_NAME  ANNEES_XP
0  2843838     Ramos      David        1.0
1  2479537     Small       Carl        2.5
2  2533337     Evans     Carmen        2.0
3  2446382   Pittman     Brandi        2.5
4  2433124    Thomas      Julie        2.5


# USer an dtheir name + experience in year is clean 

In [21]:
print(expyears.duplicated().sum())
print(expyears[expyears.duplicated(subset=['USER_ID'], keep=False)])


0
Empty DataFrame
Columns: [USER_ID, LAST_NAME, FIRST_NAME, ANNEES_XP]
Index: []


In [22]:
skillsclean = pd.read_csv("/Users/inesbenhamza/Downloads/deeplearningproject/data/raw/skillscleaned.csv")

skillsclean
#print(print(skillsclean[skillsclean.duplicated(subset=['USER_ID'], keep=True)]))



,USER_ID,SKILLS_DSC,DOMAIN_DSC,LEVEL_VAL
0,2433083,DBT,Data Integration,80.0
1,2433083,Microsoft Analytics Platform System (SSAS),Data Management,80.0
2,2433083,Microsoft Azure Data Factory,Data Integration,100.0
3,2433083,Microsoft Azure Data Lake Store,Data Management,90.0
4,2433083,Microsoft Azure SQL Database,Data Management,90.0
...,...,...,...,...
979,2978961,Snowflake Data Cloud,Data Management,0.0
980,2978961,Tableau Desktop and Online,Data Analytics,0.0
981,3072774,Python,Programming Language,40.0
982,3072774,R,Programming Language,40.0


In [23]:
skillsclean['DOMAIN_DSC'].unique()

array(['Data Integration', 'Data Management', 'Agile Delivery',
       'Financial Close', 'Data Analytics', 'Programming Language',
       'Data Platform', 'Data Governance', 'Data Science',
       'Financial Planning and Analysis'], dtype=object)

In [24]:

skills_df = skillsclean  


pivoted_df = skills_df.pivot_table(
    index='USER_ID',
    columns='DOMAIN_DSC',
    values='LEVEL_VAL',
    aggfunc=lambda x: dict(zip(skills_df.loc[x.index, 'SKILLS_DSC'], x))
)


pivoted_df = pivoted_df.reset_index()


pivoted_df

pivoted_df.to_csv("skills_pivoted.csv", index=False)

In [25]:
print(pivoted_df.head())

DOMAIN_DSC  USER_ID                                     Agile Delivery  \
0           2433083                                                NaN   
1           2433092  {'Atlassian JIRA Software': 100.0, 'Microsoft ...   
2           2433099  {'Atlassian JIRA Software': 80.0, 'Microsoft A...   
3           2433109  {'Atlassian JIRA Software': 100.0, 'Microsoft ...   
4           2433111                                                NaN   

DOMAIN_DSC                                     Data Analytics Data Governance  \
0                                                         NaN             NaN   
1                                                         NaN             NaN   
2                            {'SAP BusinessObjects BI': 20.0}             NaN   
3                                                         NaN             NaN   
4           {'Microsoft Power BI': 80.0, 'Tableau Desktop ...             NaN   

DOMAIN_DSC                                   Data Integration  \
0  

merging
with availibility 
expereince 
mission 
lang 

In [26]:
# checking the col 
pivoted_df
print(pivoted_df.columns)

Index(['USER_ID', 'Agile Delivery', 'Data Analytics', 'Data Governance',
       'Data Integration', 'Data Management', 'Data Platform', 'Data Science',
       'Financial Close', 'Financial Planning and Analysis',
       'Programming Language'],
      dtype='object', name='DOMAIN_DSC')


In [27]:

fulldf = expyears.merge(pivoted_df, on="USER_ID", how="left")
fulldf['USER_ID'].nunique()

119

# df with user id, experience so ( USER) and SKILLS 

In [28]:
fulldf

,USER_ID,LAST_NAME,FIRST_NAME,ANNEES_XP,Agile Delivery,Data Analytics,Data Governance,Data Integration,Data Management,Data Platform,Data Science,Financial Close,Financial Planning and Analysis,Programming Language
0,2843838,Ramos,David,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'Java': 70.0, 'Javascript': 20.0, 'PL/SQL': 5..."
1,2479537,Small,Carl,2.5,NaN,{'Oracle Analytics Cloud': 60.0},NaN,NaN,NaN,NaN,NaN,{'Oracle Hyperion Financial Management (HFM)':...,"{'Applied OLAP Dodeca': 80.0, 'Oracle Cloud EP...","{'Java': 60.0, 'VBA': 100.0}"
2,2533337,Evans,Carmen,2.0,{'GitLab': 40.0},NaN,{'Talend Open Studio for Data Quality': 20.0},"{'Oracle Data Integrator (ODI)': 30.0, 'Talend...",NaN,NaN,"{'Anaconda Enterprise': 0.0, 'Matlab': 0.0}",NaN,{'IBM Cognos TM1': 10.0},"{'DAX': 20.0, 'Java': 20.0, 'Python': 40.0, 'S..."
3,2446382,Pittman,Brandi,2.5,NaN,"{'Microsoft Power BI': 80.0, 'Microsoft SQL Se...",NaN,{'Microsoft SQL Server Integration Services (S...,{'Microsoft Analytics Platform System (SSAS)':...,NaN,NaN,NaN,NaN,"{'Java': 40.0, 'Python': 60.0, 'R': 40.0, 'SQL..."
4,2433124,Thomas,Julie,2.5,NaN,NaN,NaN,NaN,{'Snowflake Data Cloud': 100.0},NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,2713733,Williams,Raymond,1.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
115,2456947,Riddle,Karen,2.5,NaN,NaN,NaN,NaN,"{'IBM Db2': 80.0, 'Microsoft Analytics Platfor...",NaN,NaN,"{'CCH Tagetik CPM Conso': 60.0, 'Oracle Hyperi...","{'CCH Tagetik CPM Planning': 80.0, 'Oracle Hyp...","{'Java': 50.0, 'Python': 40.0, 'R': 40.0, 'SQL..."
116,2447672,Gonzalez,Diana,2.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
117,2789547,Williams,Brian,1.0,"{'Atlassian JIRA Software': 80.0, 'Terraform':...",{'Microsoft Power BI': 80.0},{'Azure Purview': 60.0},{'Microsoft Azure Data Factory': 80.0},NaN,"{'Databricks Lakehouse Platform': 80.0, 'Googl...",NaN,NaN,NaN,"{'Java': 30.0, 'Python': 70.0, 'SQL': 80.0, 'S..."


In [29]:
availibility= pd.read_csv('/Users/inesbenhamza/Downloads/deeplearningproject/data/HCK_HEC_STAFFING.csv')

In [30]:
fulldf = fulldf.merge(availibility, on="USER_ID", how="left")

In [31]:
fulldf

,USER_ID,LAST_NAME,FIRST_NAME,ANNEES_XP,Agile Delivery,Data Analytics,Data Governance,Data Integration,Data Management,Data Platform,...,MONTH_3,MONTH_4,MONTH_5,MONTH_6,MONTH_7,MONTH_8,MONTH_9,MONTH_10,MONTH_11,MONTH_12
0,2843838,Ramos,David,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,100,100,100,100,100,100,100,100,100,100
1,2479537,Small,Carl,2.5,NaN,{'Oracle Analytics Cloud': 60.0},NaN,NaN,NaN,NaN,...,100,100,100,100,100,100,100,100,100,100
2,2533337,Evans,Carmen,2.0,{'GitLab': 40.0},NaN,{'Talend Open Studio for Data Quality': 20.0},"{'Oracle Data Integrator (ODI)': 30.0, 'Talend...",NaN,NaN,...,25,25,25,25,25,25,25,25,25,25
3,2446382,Pittman,Brandi,2.5,NaN,"{'Microsoft Power BI': 80.0, 'Microsoft SQL Se...",NaN,{'Microsoft SQL Server Integration Services (S...,{'Microsoft Analytics Platform System (SSAS)':...,NaN,...,25,25,25,25,25,25,50,25,25,25
4,2433124,Thomas,Julie,2.5,NaN,NaN,NaN,NaN,{'Snowflake Data Cloud': 100.0},NaN,...,25,25,25,25,50,25,25,25,25,25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,2713733,Williams,Raymond,1.5,NaN,NaN,NaN,NaN,NaN,NaN,...,25,25,25,25,25,25,25,25,25,25
115,2456947,Riddle,Karen,2.5,NaN,NaN,NaN,NaN,"{'IBM Db2': 80.0, 'Microsoft Analytics Platfor...",NaN,...,0,0,0,0,0,0,0,0,0,0
116,2447672,Gonzalez,Diana,2.5,NaN,NaN,NaN,NaN,NaN,NaN,...,0,25,0,0,0,0,0,0,0,0
117,2789547,Williams,Brian,1.0,"{'Atlassian JIRA Software': 80.0, 'Terraform':...",{'Microsoft Power BI': 80.0},{'Azure Purview': 60.0},{'Microsoft Azure Data Factory': 80.0},NaN,"{'Databricks Lakehouse Platform': 80.0, 'Googl...",...,25,25,25,25,25,50,25,25,25,25


for lang and mission multiple col 

In [32]:
langskills = pd.read_csv ('/Users/inesbenhamza/Downloads/deeplearningproject/data/raw/langcleaned.csv')

lang_profiles = langskills.groupby("USER_ID").apply(
    lambda x: dict(zip(x["LANGUAGE_SKILL_DSC"], x["LANGUAGE_SKILL_LVL"]))
).reset_index(name="LANGUAGE_PROFILE")



/var/folders/ng/wz4hfzq14nggvhjg7kkdsts80000gn/T/ipykernel_50696/174774832.py:3: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  lang_profiles = langskills.groupby("USER_ID").apply(


In [33]:
fulldf = fulldf.merge(lang_profiles, on="USER_ID", how="left")

fulldf

,USER_ID,LAST_NAME,FIRST_NAME,ANNEES_XP,Agile Delivery,Data Analytics,Data Governance,Data Integration,Data Management,Data Platform,...,MONTH_4,MONTH_5,MONTH_6,MONTH_7,MONTH_8,MONTH_9,MONTH_10,MONTH_11,MONTH_12,LANGUAGE_PROFILE
0,2843838,Ramos,David,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,100,100,100,100,100,100,100,100,100,"{'English': 100.0, 'French': 100.0}"
1,2479537,Small,Carl,2.5,NaN,{'Oracle Analytics Cloud': 60.0},NaN,NaN,NaN,NaN,...,100,100,100,100,100,100,100,100,100,"{'English': 90.0, 'French': 100.0}"
2,2533337,Evans,Carmen,2.0,{'GitLab': 40.0},NaN,{'Talend Open Studio for Data Quality': 20.0},"{'Oracle Data Integrator (ODI)': 30.0, 'Talend...",NaN,NaN,...,25,25,25,25,25,25,25,25,25,"{'English': 50.0, 'French': 100.0}"
3,2446382,Pittman,Brandi,2.5,NaN,"{'Microsoft Power BI': 80.0, 'Microsoft SQL Se...",NaN,{'Microsoft SQL Server Integration Services (S...,{'Microsoft Analytics Platform System (SSAS)':...,NaN,...,25,25,25,25,25,50,25,25,25,"{'English': 60.0, 'French': 100.0}"
4,2433124,Thomas,Julie,2.5,NaN,NaN,NaN,NaN,{'Snowflake Data Cloud': 100.0},NaN,...,25,25,25,50,25,25,25,25,25,"{'English': 100.0, 'French': 100.0}"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,2713733,Williams,Raymond,1.5,NaN,NaN,NaN,NaN,NaN,NaN,...,25,25,25,25,25,25,25,25,25,"{'English': 100.0, 'French': 80.0}"
115,2456947,Riddle,Karen,2.5,NaN,NaN,NaN,NaN,"{'IBM Db2': 80.0, 'Microsoft Analytics Platfor...",NaN,...,0,0,0,0,0,0,0,0,0,"{'English': 100.0, 'French': 100.0}"
116,2447672,Gonzalez,Diana,2.5,NaN,NaN,NaN,NaN,NaN,NaN,...,25,0,0,0,0,0,0,0,0,"{'English': 80.0, 'French': 100.0}"
117,2789547,Williams,Brian,1.0,"{'Atlassian JIRA Software': 80.0, 'Terraform':...",{'Microsoft Power BI': 80.0},{'Azure Purview': 60.0},{'Microsoft Azure Data Factory': 80.0},NaN,"{'Databricks Lakehouse Platform': 80.0, 'Googl...",...,25,25,25,25,50,25,25,25,25,"{'English': 70.0, 'French': 100.0}"


In [34]:
mission = pd.read_csv('/Users/inesbenhamza/Downloads/deeplearningproject/data/HCK_HEC_XP.csv')

# here we gorup mission by user so that it become 
USER_ID	MISSIONS_LIST
2843838	[mission1, mission2, mission3, …]

In [35]:

missions_grouped = mission.groupby("USER_ID")["MISSION_DSC"].apply(list).reset_index(name="MISSIONS_LIST")


missions_grouped["MISSIONS_TEXT"] = missions_grouped["MISSIONS_LIST"].apply(lambda x: " ".join(x))

print (missions_grouped)

      USER_ID                                      MISSIONS_LIST  \
0     2432194  [Analyse de la situation actuelle et définitio...   
1     2433083  [Exhaustive data analysis, Translation of the ...   
2     2433092  [Gestion du budget., Planification de l'échéan...   
3     2433099  [Faire le suivi des activités de configuration...   
4     2433109  [Understanding of customer requirements, Analy...   
..        ...                                                ...   
119  29209385  [Coordinated clinical documentation workflows ...   
120  29209386  [Oversaw scheduling, planning, and logistics f...   
121  29209387  [Assisted regional public health authorities i...   
122  29209388  [Conducted internal audits on patient safety p...   
123  29209389  [Supported principal investigators in preparin...   

                                         MISSIONS_TEXT  
0    Analyse de la situation actuelle et définition...  
1    Exhaustive data analysis Translation of the bu...  
2    Gestion

In [36]:
fulldf = fulldf.merge(missions_grouped, on="USER_ID", how="left")


In [37]:
fulldf

,USER_ID,LAST_NAME,FIRST_NAME,ANNEES_XP,Agile Delivery,Data Analytics,Data Governance,Data Integration,Data Management,Data Platform,...,MONTH_6,MONTH_7,MONTH_8,MONTH_9,MONTH_10,MONTH_11,MONTH_12,LANGUAGE_PROFILE,MISSIONS_LIST,MISSIONS_TEXT
0,2843838,Ramos,David,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,100,100,100,100,100,100,100,"{'English': 100.0, 'French': 100.0}",[Mise en place d’un pipeline RAG pour la migra...,Mise en place d’un pipeline RAG pour la migrat...
1,2479537,Small,Carl,2.5,NaN,{'Oracle Analytics Cloud': 60.0},NaN,NaN,NaN,NaN,...,100,100,100,100,100,100,100,"{'English': 90.0, 'French': 100.0}",[Mise en œuvre d’une application de simulation...,Mise en œuvre d’une application de simulation ...
2,2533337,Evans,Carmen,2.0,{'GitLab': 40.0},NaN,{'Talend Open Studio for Data Quality': 20.0},"{'Oracle Data Integrator (ODI)': 30.0, 'Talend...",NaN,NaN,...,25,25,25,25,25,25,25,"{'English': 50.0, 'French': 100.0}","[Intégration de données, Analyser et résoudre ...",Intégration de données Analyser et résoudre le...
3,2446382,Pittman,Brandi,2.5,NaN,"{'Microsoft Power BI': 80.0, 'Microsoft SQL Se...",NaN,{'Microsoft SQL Server Integration Services (S...,{'Microsoft Analytics Platform System (SSAS)':...,NaN,...,25,25,25,50,25,25,25,"{'English': 60.0, 'French': 100.0}","[Analyser le système d'information financier, ...",Analyser le système d'information financier Co...
4,2433124,Thomas,Julie,2.5,NaN,NaN,NaN,NaN,{'Snowflake Data Cloud': 100.0},NaN,...,25,50,25,25,25,25,25,"{'English': 100.0, 'French': 100.0}",[Restitution et transmission des connaissances...,Restitution et transmission des connaissances ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
114,2713733,Williams,Raymond,1.5,NaN,NaN,NaN,NaN,NaN,NaN,...,25,25,25,25,25,25,25,"{'English': 100.0, 'French': 80.0}","[Support aux utilisateurs finaux de niveau 2, ...",Support aux utilisateurs finaux de niveau 2 Pr...
115,2456947,Riddle,Karen,2.5,NaN,NaN,NaN,NaN,"{'IBM Db2': 80.0, 'Microsoft Analytics Platfor...",NaN,...,0,0,0,0,0,0,0,"{'English': 100.0, 'French': 100.0}",[Analyses des fichiers Excel utilisés pour aut...,Analyses des fichiers Excel utilisés pour auto...
116,2447672,Gonzalez,Diana,2.5,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,"{'English': 80.0, 'French': 100.0}",[**Business Analyst for the implementation of ...,**Business Analyst for the implementation of a...
117,2789547,Williams,Brian,1.0,"{'Atlassian JIRA Software': 80.0, 'Terraform':...",{'Microsoft Power BI': 80.0},{'Azure Purview': 60.0},{'Microsoft Azure Data Factory': 80.0},NaN,"{'Databricks Lakehouse Platform': 80.0, 'Googl...",...,25,25,50,25,25,25,25,"{'English': 70.0, 'French': 100.0}",[Design and implementation of data architectur...,Design and implementation of data architecture...


In [38]:
fulldf.to_csv("full_df.csv", index=False)

# fine tuning 

In [39]:
import pandas as pd
import os 

In [40]:
if 'project_root' not in globals():
    current_dir = os.getcwd()
    if os.path.basename(current_dir) == 'notebooks':
        project_root = os.path.dirname(current_dir)
    else:
        project_root = current_dir

data_dir = os.path.join(project_root, "data")
print(os.listdir(data_dir))

full_df = pd.read_csv(
    os.path.join(project_root, "full_df.csv"),
    sep=',', 
    on_bad_lines='skip', 
    encoding='utf-8'
)

['HCK_HEC_LANG.csv', '.DS_Store', 'HCK_HEC_XP.csv', 'HCK_HEC_SKILLS.csv', 'HCK_HEC_USER.csv', 'HCK_HEC_STAFFING.csv', 'raw']


In [41]:
full_df["USER_ID"].nunique()

128

fine tuning bert for paraphrase detection (are the two sentences paraphrases of each other?)

In [42]:
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader

In [43]:
#!pip install --upgrade datasets sentence-transformers

In [44]:
import sys
from datasets import Dataset
sys.modules['datasets'].Dataset = Dataset

# language finteuning

In [ ]:
model = SentenceTransformer("paraphrase-multilingual-mpnet-base-v2")

# Fine-tuning for English-French translation matching across all job categories
# Positive pairs: English sentences matched with their French equivalents for each job role

# DATA ENGINEER - English-French pairs
data_engineer_pairs = [
    ("Design and develop robust and optimized data ingestion pipelines using Azure Data Factory",
     "Concevoir et développer des pipelines d'ingestion de données robustes et optimisés à l'aide d'Azure Data Factory"),
    ("Develop complex transformations using Apache Spark (Scala/Python) in Azure Databricks",
     "Développer des transformations complexes en utilisant Apache Spark (Scala/Python) dans Azure Databricks"),
    ("Implement streaming and batch data flows via Databricks Streaming and Apache Kafka",
     "Implémenter des flux de données streaming et batch via Databricks Streaming et Apache Kafka"),
    ("Created data pipelines using ADF",
     "Concevoir des pipelines d'ingestion de données avec Azure Data Factory"),
    ("Participate in defining and implementing data models adapted to analytical needs",
     "Participer à la définition et à la mise en œuvre de modèles de données adaptés aux besoins analytiques"),
    ("Implement data quality validation processes",
     "Mettre en œuvre des processus de validation de la qualité des données"),
    ("Automate pipeline deployments and tests via Azure DevOps",
     "Automatiser les déploiements et les tests des pipelines via Azure DevOps"),
    ("Document developed solutions according to internal standards",
     "Documenter les solutions développées selon les normes internes"),
    ("Collaborate with Data Science, BI, and IT Infrastructure teams",
     "Travailler en collaboration avec les équipes Data Science, BI et IT Infrastructure"),
    ("Contribute to the development of data architecture and data governance principles",
     "Participer à l'élaboration des principes d'architecture data et de gouvernance des données"),
    ("Created data pipelines using ADF", "Concevoir des pipelines d’ingestion de données avec Azure Data Factory"),
    ("Develop complex transformations using Apache Spark", "Développer des transformations complexes avec Apache Spark"),
    ("Developed PySpark jobs on Databricks", "Créer des transformations complexes avec Spark dans Azure Databricks"),
    ("Monitored and maintained ADF pipelines in production", "Assurer la qualité et la fiabilité des pipelines dans Azure Data Factory"),
    ("Improved data quality checks in Spark pipelines", "Mettre en œuvre des processus de validation de la qualité des données"),
    ("Utilisation de Synapse serverless et création de workspace", "Développer des transformations complexes en utilisant Apache Spark (Scala/Python) dans Azure Databricks"),
    ("Utilisation de Azure DevOps pour gérer les environnements", "Automatiser les déploiements et les tests des pipelines via Azure DevOps"),
    ("Developed complex transformations using Apache Spark", "Développer des transformations complexes avec Apache Spark"),
    ("Implemented streaming and batch data flows via Databricks", "Implémenter des flux de données streaming et batch via Databricks Streaming"),
    ("Automate pipeline deployments and tests via Azure DevOps", "Automatiser les déploiements et les tests des pipelines via Azure DevOps"),
    ("Contribute to the development of data architecture and data governance principles", "Contribuer à l'élaboration des principes d'architecture data et de gouvernance des données"),
    ("Built CI/CD workflows using Azure DevOps", "Automatiser les déploiements via Azure DevOps (CI/CD)"),
    ("Designed a lakehouse architecture", "Participer à l’élaboration de l’architecture data"),
    ("Protected team focus from outside distractions", "Protéger l’équipe des interférences externes"),
    ("Création de modèles de données en étoile avec gestion des SCD2", "Participer à la définition et à la mise en œuvre de modèles de données adaptés aux besoins analytiques"),
    ("Mise en place de Snowflake, gestion des droits, objets, dynamic tables", "Contribuer à l’élaboration des principes d’architecture data et de gouvernance des données"),
    ("Réalisation d’analyses de performance sur Snowflake", "Mettre en œuvre des processus de validation de la qualité des données"),
    ("Performed data integration from CRM and Google Analytics", "Intégrer les données issues de diverses plateformes (Google Analytics, CRM...)"),
    ("Designed data models aligned with business goals", "Concevoir des modèles de données adaptés aux objectifs métiers"),
    ("Analyzed sales trends and generated insights for decision making", "Analyser les tendances de vente et générer des insights pour la prise de décision"),
    ("Removed impediments blocking team progress", "Supprimer les obstacles bloquant la progression de l'équipe"),
    ("Participate in defining and implementing data models", "Participer à la définition et à la mise en œuvre de modèles de données adaptés aux besoins analytiques"),
    ("Implement data quality validation processes", "Mettre en œuvre des processus de validation de la qualité des données"),
    ("Document developed solutions according to internal standards", "Documenter les solutions développées selon les normes internes"),
    ("Collaborate with Data Science, BI, and IT Infrastructure teams", "Travailler en collaboration avec les équipes Data Science, BI et IT Infrastructure"),
    ("Intégration de données depuis des fichiers Excel et des systèmes externes", "Intégrer les données issues de diverses plateformes (Google Analytics, CRM...)"),
]

# SCRUM MASTER - English-French pairs
scrum_master_pairs = [
    ("Facilitated Scrum ceremonies including planning and retrospectives",
     "Faciliter l'ensemble des cérémonies Scrum"),
    ("Acted as Agile coach for cross-functional team",
     "Agir en tant que coach Agile"),
    ("Helped Product Owner manage backlog and define priorities",
     "Aider le Product Owner à gérer efficacement le backlog produit"),
    ("Tracked team velocity and analyzed sprint metrics",
     "Mesurer et analyser la performance de l'équipe via des indicateurs clés"),
    ("Protected team from external distractions",
     "Protéger l'équipe des interférences externes"),
    ("Organized daily standup meetings and sprint planning sessions",
     "Organiser les réunions quotidiennes et les sessions de planification de sprint"),
    ("Removed impediments blocking team progress",
     "Supprimer les obstacles bloquant la progression de l'équipe"),
    ("Coached team on Agile methodologies and best practices",
     "Coacher l'équipe sur les méthodologies Agile et les meilleures pratiques"),
    ("Helped PO with backlog prioritization", "Aider le Product Owner à gérer efficacement le backlog produit"),
    ("Facilitated daily standups and retrospectives", "Animer les cérémonies Scrum (daily, rétrospective, sprint planning)"),
    ("Facilitation des cérémonies Agile (Daily, Rétro, Planning)", "Faciliter l’ensemble des cérémonies Scrum"),
    ("Accompagnement des équipes pour adopter les pratiques Agile", "Agir en tant que coach Agile"),
    ("Soutien au Product Owner dans la priorisation du backlog", "Aider le Product Owner à gérer efficacement le backlog produit"),
    ("Tracked team velocity and analyzed sprint metrics", "Mesurer et analyser la performance de l'équipe via des indicateurs clés"),
    ("Coached team on Agile methodologies and best practices", "Coacher l'équipe sur les méthodologies Agile et les meilleures pratiques"),
    ("Organized daily standup meetings and sprint planning sessions", "Organiser les réunions quotidiennes et les sessions de planification de sprint"),
]

# DATA ANALYST - English-French pairs
data_analyst_pairs = [
    ("Built dashboards in Power BI",
     "Créer des tableaux de bord Power BI"),
    ("Created interactive reports and visualizations for business stakeholders",
     "Créer des rapports interactifs et des visualisations pour les parties prenantes"),
    ("Wrote performant SQL queries to transform marketing data",
     "Rédiger du code SQL performant"),
    ("Performed data integration from CRM and Google Analytics",
     "Intégrer les données issues de diverses plateformes (Google Analytics, CRM...)"),
    ("Collaborated with analysts to understand reporting needs",
     "Collaborer étroitement avec les analystes pour comprendre leurs besoins"),
    ("Designed data models aligned with business goals",
     "Concevoir des modèles de données adaptés aux objectifs métiers"),
    ("Analyzed sales trends and generated insights for decision making",
     "Analyser les tendances de vente et générer des insights pour la prise de décision"),
    ("Created KPI dashboards to track business performance",
     "Créer des tableaux de bord KPI pour suivre la performance de l'entreprise"),
    ("Built dashboards in Power BI", "Créer des tableaux de bord Power BI"),
    ("Worked closely with stakeholders to define KPIs and dashboards", "Collaborer avec les analystes pour concevoir des tableaux de bord métier"),
    ("Wrote optimized SQL for analytics reporting", "Rédiger du code SQL performant pour l’analyse de données"),
    ("Worked with BI and data science to align reporting", "Travailler en collaboration avec les équipes Data Science, BI et IT"),
    ("Analyse des KPIs projet et amélioration continue", "Mesurer et analyser la performance de l’équipe via des indicateurs clés"),
    ("Prepared financial reports and budget variance analysis", "Préparer les rapports financiers et l'analyse des écarts budgétaires"),
    ("Préparation de rapports financiers et analyse des écarts", "Prepared financial reports and budget variance analysis"),
    ("Created interactive reports and visualizations for business stakeholders", "Créer des rapports interactifs et des visualisations pour les parties prenantes"),
    ("Wrote performant SQL queries to transform marketing data", "Rédiger du code SQL performant pour transformer les données marketing"),
    ("Created KPI dashboards to track business performance", "Créer des tableaux de bord KPI pour suivre la performance de l'entreprise"),
    ("Collaborated with analysts to understand reporting needs", "Collaborer étroitement avec les analystes pour comprendre leurs besoins"),
    ("Création de rapports Power BI à partir de sources SQL Server", "Développer des tableaux de bord et rapports dans Power BI"),
    ("Écriture de requêtes SQL complexes pour extraire des indicateurs clés", "Rédiger du code SQL performant"),
    ("Collaboration avec les analystes pour définir des rapports pertinents", "Collaborer étroitement avec les analystes pour comprendre leurs besoins"),
]

# HEALTHCARE - English-French pairs
healthcare_pairs = [
    ("Analyzed patient medical records to establish accurate diagnosis",
     "Analyser les dossiers médicaux des patients pour établir un diagnostic précis"),
    ("Coordinated care with medical team and healthcare professionals",
     "Coordonner les soins avec l'équipe médicale et les professionnels de santé"),
    ("Implemented patient record management system and clinical treatments",
     "Mettre en place un système de gestion des dossiers patients et des traitements cliniques"),
    ("Assisted physicians in patient care and treatment planning",
     "Assister les médecins dans les soins aux patients et la planification des traitements"),
    ("Managed clinical documentation and medical records",
     "Gérer la documentation clinique et les dossiers médicaux"),
    ("Collaborated with nursing staff on patient care protocols",
     "Collaborer avec le personnel infirmier sur les protocoles de soins aux patients"),
    ("Analyzed patient medical records to establish accurate diagnosis", "Analyser les dossiers médicaux des patients pour établir un diagnostic précis"),
    ("Coordinated care with medical team and healthcare professionals", "Coordonner les soins avec l'équipe médicale et les professionnels de santé"),
    ("Implemented patient record management system and clinical treatments", "Mettre en place un système de gestion des dossiers patients et des traitements cliniques"),
    ("Assisted physicians in patient care and treatment planning", "Assister les médecins dans les soins aux patients et la planification des traitements"),
    ("Managed clinical documentation and medical records", "Gérer la documentation clinique et les dossiers médicaux"),
    ("Collaborated with nursing staff on patient care protocols", "Collaborer avec le personnel infirmier sur les protocoles de soins aux patients"),
    ("Conducted patient assessments and health screenings", "Réaliser des évaluations de patients et des dépistages de santé"),
    ("Monitored patient vital signs and medication administration", "Surveiller les signes vitaux des patients et l'administration de médicaments"),
    ("Analyse des dossiers médicaux pour établir un diagnostic", "Analyzed patient medical records to establish accurate diagnosis"),
    ("Coordination des soins avec l'équipe médicale", "Coordinated care with medical team and healthcare professionals"),
    ("Gestion des dossiers patients et traitements cliniques", "Managed patient records and clinical treatment protocols"),
    ("Assistance aux médecins dans les soins aux patients", "Assisted physicians in patient care and treatment planning"),
]

# FINANCE - English-French pairs
finance_pairs = [
    ("Implemented budget simulation application and rolling forecast",
     "Mise en œuvre d'une application de simulation budgétaire et de rolling forecast"),
    ("Optimized financial close processes and budget planning",
     "Optimisation des processus de clôture financière et de planification budgétaire"),
    ("Designed and implemented cost allocation application for monthly forecast process",
     "Conception et mise en œuvre d'une application d'allocation de coûts pour le processus de forecast mensuel"),
    ("Conducted financial audits and compliance reviews",
     "Réaliser des audits financiers et des revues de conformité"),
    ("Prepared financial reports and budget variance analysis",
     "Préparer les rapports financiers et l'analyse des écarts budgétaires"),
    ("Managed financial planning and analysis activities",
     "Gérer les activités de planification et d'analyse financières"),
    ("Implemented budget simulation application and rolling forecast", "Mise en œuvre d'une application de simulation budgétaire et de rolling forecast"),
    ("Optimized financial close processes and budget planning", "Optimisation des processus de clôture financière et de planification budgétaire"),
    ("Designed and implemented cost allocation application for monthly forecast process", "Conception et mise en œuvre d'une application d'allocation de coûts pour le processus de forecast mensuel"),
    ("Conducted financial audits and compliance reviews", "Réaliser des audits financiers et des revues de conformité"),
    ("Managed financial planning and analysis activities", "Gérer les activités de planification et d'analyse financières"),
    ("Developed financial models for forecasting and budgeting", "Développer des modèles financiers pour la prévision et la budgétisation"),
    ("Analyzed financial data to support strategic decision making", "Analyser les données financières pour soutenir la prise de décision stratégique"),
    ("Mise en œuvre d'une application de simulation budgétaire", "Implemented budget simulation application and rolling forecast"),
    ("Optimisation des processus de clôture financière", "Optimized financial close processes and budget planning"),
    ("Réalisation d'audits financiers et revues de conformité", "Conducted financial audits and compliance reviews"),
    ("Gestion de la planification et analyse financières", "Managed financial planning and analysis activities"),
]


positive_pairs = (
    data_engineer_pairs +
    scrum_master_pairs +
    data_analyst_pairs +
    healthcare_pairs +
    finance_pairs
)


negative_pairs = [
    ("Built dashboards in Power BI", "Préparer des repas en cuisine"),
    ("Created REST APIs", "Aider les patients dans les soins"),
    ("Facilitated Scrum ceremonies", "Développer des pipelines ETL"),
    ("Analyzed patient medical records", "Optimiser les processus de clôture financière"),
    ("Implemented budget simulation", "Faciliter les cérémonies Scrum"),
    ("Developed PySpark jobs", "Coordonner les soins avec l'équipe médicale"),
    ("Automated deployment with Azure DevOps", "Gérer les présences dans un club de sport"),
    ("Helped define data architecture", "Traduire un document marketing en espagnol"),
    ("Documented pipelines", "Rédiger une lettre de motivation"),
    ("Conducted financial audits", "Agir en tant que coach Agile"),
    ("Developed PySpark jobs", "Animer un atelier marketing"),
    ("Implemented Kafka streaming", "Créer du contenu pour les réseaux sociaux"),
    ("Documented pipelines", "Traduire un document marketing"),
    ("Designed KPI dashboards", "Agir en tant que coach Agile (Scrum Master)"),
    ("Helped organize hackathon logistics", "Construire des pipelines avec Apache Spark"),
    ("Création de dashboards dans Power BI", "Automatiser les déploiements et les tests des pipelines via Azure DevOps"),
    ("Faciliter les cérémonies Scrum", "Développer des transformations complexes en utilisant Apache Spark (Scala/Python) dans Azure Databricks"),
    ("Élaboration de reporting mensuel", "Documenter les solutions développées selon les normes internes"),
    ("Écriture de requêtes SQL pour KPIs marketing", "Contribuer à l’élaboration des principes d’architecture data et de gouvernance des données"),
    ("Built dashboards in Power BI", "Cuisiner un repas gastronomique"),
    ("Developed PySpark jobs", "Enseigner les mathématiques à des étudiants"),
    ("Facilitated Scrum ceremonies", "Réparer une voiture en panne"),
    ("Analyzed patient medical records", "Créer une œuvre d'art contemporain"),
    ("Implemented budget simulation", "Organiser un événement de mariage"),
    ("Created data pipelines", "Traduire un livre de poésie"),
    ("Monitored ADF pipelines", "Diriger un orchestre symphonique"),
    ("Wrote SQL queries", "Peindre un tableau à l'huile"),
    ("Helped PO with backlog", "Concevoir une collection de mode"),
    ("Coordinated care with medical team", "Développer une application mobile de jeu"),
    ("Conducted financial audits", "Écrire un scénario de film"),
    ("Prepared financial reports", "Créer une chorégraphie de danse"),
]

train_examples = [
    InputExample(texts=[text1, text2], label=1.0) for text1, text2 in positive_pairs
] + [
    InputExample(texts=[text1, text2], label=0.0) for text1, text2 in negative_pairs
]


def collate_fn(batch):
    
    return batch

train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=16, collate_fn=collate_fn)  # Increased from 1


train_loss = losses.ContrastiveLoss(
    model,
    distance_metric=losses.SiameseDistanceMetric.COSINE_DISTANCE,
    margin=0.5  
)

print(f"Training with {len(train_examples)} examples")
print(f"Positive pairs: {len(positive_pairs)}, Negative pairs: {len(negative_pairs)}")
print(f"  - Data Engineer pairs: {len(data_engineer_pairs)}")
print(f"  - Scrum Master pairs: {len(scrum_master_pairs)}")
print(f"  - Data Analyst pairs: {len(data_analyst_pairs)}")
print(f"  - Healthcare pairs: {len(healthcare_pairs)}")
print(f"  - Finance pairs: {len(finance_pairs)}")
print(f"Using ContrastiveLoss with margin=0.5")
print("This fine-tuning will make English sentences match their French equivalents across all job categories")

import sys

if 'datasets' not in sys.modules:
    import datasets
    sys.modules['datasets'] = datasets

import torch
from torch.optim import AdamW
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer
from tqdm import tqdm

optimizer = AdamW(model.parameters(), lr=2e-5)

import torch
device = torch.device('cpu') #no gpu
model = model.to(device)

model.train()
num_epochs = 5
warmup_steps = 100
total_steps = len(train_dataloader) * num_epochs
current_step = 0

print(f"Starting training for {num_epochs} epochs...")
print(f"Total steps: {total_steps}")

for epoch in range(num_epochs):
    print(f"\nEpoch {epoch + 1}/{num_epochs}")
    epoch_loss = 0
    
    for batch_idx, batch in enumerate(tqdm(train_dataloader, desc=f"Epoch {epoch + 1}")):
        batch_losses = []
        
        for example in batch:

            text1, text2 = example.texts
            label = example.label
            
            features1 = model.tokenize([text1])
            features2 = model.tokenize([text2])
            
            features1 = {k: v.to(device) if isinstance(v, torch.Tensor) else v for k, v in features1.items()}
            features2 = {k: v.to(device) if isinstance(v, torch.Tensor) else v for k, v in features2.items()}
            
            sentence_features = [features1, features2]
            labels = torch.tensor([label], dtype=torch.float32, device=device)
            
            loss = train_loss(sentence_features, labels)
            
            batch_losses.append(loss)
        
        if batch_losses:
            avg_batch_loss = sum(batch_losses) / len(batch_losses)
            

            optimizer.zero_grad()
            avg_batch_loss.backward()
            optimizer.step()
            
            step = current_step + batch_idx
            if step < warmup_steps:
                lr_scale = min(1.0, float(step + 1) / warmup_steps)
                for pg in optimizer.param_groups:
                    pg['lr'] = 2e-5 * lr_scale
            
            epoch_loss += avg_batch_loss.item()
    
    avg_loss = epoch_loss / len(train_dataloader)
    print(f"Average loss: {avg_loss:.4f}")


model.save("modelfinetuned1")



Training with 140 examples
Positive pairs: 109, Negative pairs: 31
  - Data Engineer pairs: 36
  - Scrum Master pairs: 16
  - Data Analyst pairs: 22
  - Healthcare pairs: 18
  - Finance pairs: 17
Using ContrastiveLoss with margin=0.5
This fine-tuning will make English sentences match their French equivalents across all job categories
Starting training for 5 epochs...
Total steps: 45

Epoch 1/5


Epoch 1: 100%|██████████| 9/9 [00:41<00:00,  4.60s/it]


Average loss: 0.0163

Epoch 2/5


Epoch 2: 100%|██████████| 9/9 [00:36<00:00,  4.05s/it]


Average loss: 0.0130

Epoch 3/5


...

Average loss: 0.0111

Epoch 4/5


Epoch 4: 100%|██████████| 9/9 [00:36<00:00,  4.04s/it]


Average loss: 0.0092

Epoch 5/5


Epoch 5: 100%|██████████| 9/9 [00:36<00:00,  4.10s/it]


Average loss: 0.0082

Training complete!


# further finetuning

In [48]:
#import sys
#import subprocess

#subprocess.check_call([
    #sys.executable, "-m", "pip", "install", "accelerate>=0.26.0"
#])

In [49]:
#!pip install 'accelerate>=0.26.0'

In [ ]:
from sentence_transformers import SentenceTransformer, util
import os



orig_model = SentenceTransformer( "paraphrase-multilingual-mpnet-base-v2")
finetuned_model = SentenceTransformer("modelfinetuned1")

def sim(model, t1, t2):
    return util.pytorch_cos_sim(
        model.encode(t1, convert_to_tensor=True),
        model.encode(t2, convert_to_tensor=True)
    ).item()

en_fr_pairs = [
    ("Built dashboards in Power BI", "Créer des tableaux de bord Power BI"),
    ("Developed PySpark jobs on Databricks", "Créer des transformations complexes avec Spark dans Azure Databricks"),
    ("Facilitated daily standups and retrospectives", "Animer les cérémonies Scrum"),
    ("Analyzed patient medical records to establish accurate diagnosis", "Analyser les dossiers médicaux des patients pour établir un diagnostic précis"),
    ("Implemented budget simulation application", "Mise en œuvre d'une application de simulation budgétaire"),
    ("Created interactive reports and visualizations for business stakeholders", "Créer des rapports interactifs et des visualisations pour les parties prenantes"),
    ("Wrote performant SQL queries to transform marketing data", "Rédiger du code SQL performant pour transformer les données marketing"),
    ("Performed data integration from CRM and Google Analytics", "Intégrer les données issues de diverses plateformes (Google Analytics, CRM...)"),
    ("Designed data models aligned with business goals", "Concevoir des modèles de données adaptés aux objectifs métiers"),
    ("Helped PO with backlog prioritization", "Aider le Product Owner à gérer efficacement le backlog produit"),
    ("Tracked team velocity and analyzed sprint metrics", "Mesurer et analyser la performance de l'équipe via des indicateurs clés"),
    ("Removed impediments blocking team progress", "Supprimer les obstacles bloquant la progression de l'équipe"),
    ("Coached team on Agile methodologies and best practices", "Coacher l'équipe sur les méthodologies Agile et les meilleures pratiques"),
    ("Assisted physicians in patient care and treatment planning", "Assister les médecins dans les soins aux patients et la planification des traitements"),
    ("Managed clinical documentation and medical records", "Gérer la documentation clinique et les dossiers médicaux"),
    ("Collaborated with nursing staff on patient care protocols", "Collaborer avec le personnel infirmier sur les protocoles de soins aux patients"),
    ("Conducted patient assessments and health screenings", "Réaliser des évaluations de patients et des dépistages de santé"),
    ("Designed and implemented cost allocation application", "Conception et mise en œuvre d'une application d'allocation de coûts"),
    ("Prepared financial reports and budget variance analysis", "Préparer les rapports financiers et l'analyse des écarts budgétaires"),
    ("Developed financial models for forecasting and budgeting", "Développer des modèles financiers pour la prévision et la budgétisation"),
    ("Analyzed financial data to support strategic decision making", "Analyser les données financières pour soutenir la prise de décision stratégique"),
    ("Created data pipelines using ADF", "Concevoir des pipelines d'ingestion de données avec Azure Data Factory"),
    ("Monitored and maintained ADF pipelines in production", "Assurer la qualité et la fiabilité des pipelines dans Azure Data Factory"),
    ("Worked closely with stakeholders to define KPIs and dashboards", "Collaborer avec les analystes pour concevoir des tableaux de bord métier"),
    ("Improved data quality checks in Spark pipelines", "Mettre en œuvre des processus de validation de la qualité des données"),
    ("Built CI/CD workflows using Azure DevOps", "Automatiser les déploiements via Azure DevOps (CI/CD)"),
    ("Designed a lakehouse architecture", "Participer à l'élaboration de l'architecture data"),
    ("Worked with BI and data science to align reporting", "Travailler en collaboration avec les équipes Data Science, BI et IT"),
    ("Protected team focus from outside distractions", "Protéger l'équipe des interférences externes"),
    ("Organized daily standup meetings and sprint planning sessions", "Organiser les réunions quotidiennes et les sessions de planification de sprint"),
    ("Removed impediments blocking team progress", "Supprimer les obstacles bloquant la progression de l'équipe"),
    ("Coached team on Agile methodologies and best practices", "Coacher l'équipe sur les méthodologies Agile et les meilleures pratiques"),
    ("Created KPI dashboards to track business performance", "Créer des tableaux de bord KPI pour suivre la performance de l'entreprise"),
    ("Collaborated with analysts to understand reporting needs", "Collaborer étroitement avec les analystes pour comprendre leurs besoins"),
    ("Analyzed sales trends and generated insights for decision making", "Analyser les tendances de vente et générer des insights pour la prise de décision"),
    ("Coordinated care with medical team and healthcare professionals", "Coordonner les soins avec l'équipe médicale et les professionnels de santé"),
    ("Implemented patient record management system and clinical treatments", "Mettre en place un système de gestion des dossiers patients et des traitements cliniques"),
    ("Monitored patient vital signs and medication administration", "Surveiller les signes vitaux des patients et l'administration de médicaments"),
    ("Optimized financial close processes and budget planning", "Optimisation des processus de clôture financière et de planification budgétaire"),
    ("Conducted financial audits and compliance reviews", "Réaliser des audits financiers et des revues de conformité"),
    ("Managed financial planning and analysis activities", "Gérer les activités de planification et d'analyse financières"),
    ("Developed complex transformations using Apache Spark", "Développer des transformations complexes avec Apache Spark"),
    ("Implemented streaming and batch data flows via Databricks", "Implémenter des flux de données streaming et batch via Databricks Streaming"),
    ("Participate in defining and implementing data models", "Participer à la définition et à la mise en œuvre de modèles de données adaptés aux besoins analytiques"),
    ("Implement data quality validation processes", "Mettre en œuvre des processus de validation de la qualité des données"),
    ("Automate pipeline deployments and tests via Azure DevOps", "Automatiser les déploiements et les tests des pipelines via Azure DevOps"),
    ("Document developed solutions according to internal standards", "Documenter les solutions développées selon les normes internes"),
    ("Collaborate with Data Science, BI, and IT Infrastructure teams", "Travailler en collaboration avec les équipes Data Science, BI et IT Infrastructure"),
    ("Contribute to the development of data architecture and data governance principles", "Contribuer à l'élaboration des principes d'architecture data et de gouvernance des données"),
    ("Implémentation et automatisation sur Azure Data Factory (ADF)", "Concevoir et développer des pipelines d'ingestion de données robustes et optimisés à l'aide d'Azure Data Factory"),
    ("Utilisation de Synapse serverless et création de workspace", "Développer des transformations complexes en utilisant Apache Spark (Scala/Python) dans Azure Databricks"),
    ("Création de modèles de données en étoile avec gestion des SCD2", "Participer à la définition et à la mise en œuvre de modèles de données adaptés aux besoins analytiques"),
    ("Mise en place de Snowflake, gestion des droits, objets, dynamic tables", "Contribuer à l'élaboration des principes d'architecture data et de gouvernance des données"),
    ("Réalisation d'analyses de performance sur Snowflake", "Mettre en œuvre des processus de validation de la qualité des données"),
    ("Facilitation des cérémonies Agile (Daily, Rétro, Planning)", "Faciliter l'ensemble des cérémonies Scrum"),
    ("Accompagnement des équipes pour adopter les pratiques Agile", "Agir en tant que coach Agile"),
    ("Soutien au Product Owner dans la priorisation du backlog", "Aider le Product Owner à gérer efficacement le backlog produit"),
    ("Analyse des KPIs projet et amélioration continue", "Mesurer et analyser la performance de l'équipe via des indicateurs clés"),
]

# Wrong translations (should have LOW similarity - model should push these apart)
wrong_translations = [
    # Cross-domain wrong translations
    ("Design and develop robust data pipelines using Azure Data Factory",
     "Créer des tableaux de bord Power BI"),  # Wrong: should be about pipelines, not dashboards
    ("Facilitated Scrum ceremonies including planning and retrospectives",
     "Analyser les dossiers médicaux des patients"),  # Wrong: should be about Scrum, not healthcare
    ("Built dashboards in Power BI",
     "Mise en œuvre d'une application de simulation budgétaire"),  # Wrong: should be about dashboards, not finance
    ("Analyzed patient medical records to establish accurate diagnosis",
     "Faciliter l'ensemble des cérémonies Scrum"),  # Wrong: should be about healthcare, not Scrum
    ("Implemented budget simulation application and rolling forecast",
     "Concevoir et développer des pipelines de données"),  # Wrong: should be about finance, not data engineering
    # Completely unrelated
    ("Developed PySpark jobs on Databricks",
     "Préparer des repas en cuisine"),  # Completely unrelated
    ("Created data pipelines using ADF",
     "Organiser un événement de mariage"),  # Completely unrelated
    ("Facilitated daily standups and retrospectives",
     "Cuisiner un repas gastronomique"),  # Completely unrelated
]



def avg(lst): return sum(lst) / len(lst)


orig_trans = [sim(orig_model, a, b) for a, b in en_fr_pairs]
ft_trans   = [sim(finetuned_model, a, b) for a, b in en_fr_pairs]


orig_trans_avg = avg(orig_trans); ft_trans_avg = avg(ft_trans)
delta = ft_trans_avg - orig_trans_avg
pct = delta / orig_trans_avg * 100 if orig_trans_avg > 0 else 0
print(f"Original avg:  {orig_trans_avg:.3f}")
print(f"Finetuned avg:{ft_trans_avg:.3f}")
print(f"Delta: {delta:+.3f} ({pct:+.1f}%)")
print("Pairs (orig -> finetuned):")
for (a, b), o, f in zip(en_fr_pairs, orig_trans, ft_trans):
    d = f - o
    pct = (d / o * 100) if o > 0 else 0
    print(f"- {a[:45]}...: {o:.3f} -> {f:.3f} ({d:+.3f}, {pct:+.1f}%)")

print("\n" + "="*70)
print("WRONG TRANSLATIONS (should have LOW similarity)")
print("="*70)
orig_wrong = [sim(orig_model, a, b) for a, b in wrong_translations]
ft_wrong = [sim(finetuned_model, a, b) for a, b in wrong_translations]

orig_wrong_avg = avg(orig_wrong)
ft_wrong_avg = avg(ft_wrong)
wrong_delta = ft_wrong_avg - orig_wrong_avg
wrong_pct = wrong_delta / orig_wrong_avg * 100 if orig_wrong_avg > 0 else 0

print(f"Original Model Average:     {orig_wrong_avg:.3f}")
print(f"Fine-tuned Model Average:   {ft_wrong_avg:.3f}")
print(f"Change:                     {wrong_delta:+.3f} ({wrong_pct:+.1f}%)")
print("(Lower is better - model should push wrong translations apart)")
print("\nIndividual wrong pairs (orig -> finetuned):")
for (a, b), o, f in zip(wrong_translations, orig_wrong, ft_wrong):
    d = f - o
    pct = (d / o * 100) if o > 0 else 0
    status = "BETTER" if f < o else "WORSE" if f > o else "SAME"
    print(f"  {status} | {o:.3f} -> {f:.3f} ({d:+.3f}, {pct:+.1f}%) | {a[:40]}... vs {b[:40]}...")


print("SUMMARY")

print(f"Correct translations: {orig_trans_avg:.3f} -> {ft_trans_avg:.3f} ({delta:+.3f}, {pct:+.1f}%)")
print(f"Wrong translations:   {orig_wrong_avg:.3f} -> {ft_wrong_avg:.3f} ({wrong_delta:+.3f}, {wrong_pct:+.1f}%)")
print(f"\nGap (correct - wrong): Original: {orig_trans_avg - orig_wrong_avg:.3f}, Fine-tuned: {ft_trans_avg - ft_wrong_avg:.3f}")
gap_improvement = (ft_trans_avg - ft_wrong_avg) - (orig_trans_avg - orig_wrong_avg)
print(f"Gap improvement: {gap_improvement:+.3f} (larger gap = better discrimination)")



The tokenizer you are loading from 'modelfinetuned1' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.


Original avg:  0.794
Finetuned avg:0.881
Delta: +0.087 (+11.0%)
Pairs (orig -> finetuned):
- Built dashboards in Power BI...: 0.766 -> 0.894 (+0.128, +16.7%)
- Developed PySpark jobs on Databricks...: 0.551 -> 0.757 (+0.206, +37.4%)
- Facilitated daily standups and retrospectives...: 0.430 -> 0.665 (+0.235, +54.6%)
- Analyzed patient medical records to establish...: 0.919 -> 0.953 (+0.034, +3.7%)
- Implemented budget simulation application...: 0.493 -> 0.711 (+0.218, +44.1%)
- Created interactive reports and visualization...: 0.861 -> 0.931 (+0.070, +8.1%)
- Wrote performant SQL queries to transform mar...: 0.919 -> 0.947 (+0.028, +3.0%)
- Performed data integration from CRM and Googl...: 0.858 -> 0.910 (+0.052, +6.1%)
- Designed data models aligned with business go...: 0.845 -> 0.929 (+0.083, +9.9%)
- Helped PO with backlog prioritization...: 0.673 -> 0.873 (+0.200, +29.7%)
- Tracked team velocity and analyzed sprint met...: 0.612 -> 0.787 (+0.175, +28.6%)
- Removed impediments blocki